# Deploy Background Cleanup: Background Cleaup </font> Model Package from AWS Marketplace 


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

---

This sample notebook shows you how to deploy <font color='red'> Background Removal Model</font> using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page 
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
model_package_arn = "arn:aws:sagemaker:us-east-1:728413774894:model-package/awsmpbgcleanup"

In [21]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import numpy as np

In [23]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

'sagemaker-us-east-1-728413774894'

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [35]:
model_name = "bg-cleanup"

content_type = "application/json"

real_time_inference_instance_type = (
    "ml.m4.xlarge"
)
batch_transform_inference_instance_type = (
    "ml.m4.xlarge"
)

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [51]:
import os
import json
import base64

input_img_pth = "data/input/real-time/image.jpg" # Path to image 

# Read the image file and encode it to base64
with open(input_img_pth, "rb") as img:
    image = base64.b64encode(img.read())
    img_base64 = image.decode()

# Prepare the data in the expected format
data = {"image": img_base64}

# Write the data to a JSON file
file_name = "data/input/real-time/input.json"
with open(file_name, "w") as json_file:
    json.dump(data, json_file)

# Set the necessary parameters for invoking the SageMaker endpoint
# model_name = "model"
output_file_name = "data/output/output.txt"
content_type = "application/json"
region = "us-east-1"  # Replace with your actual AWS region, e.g., "us-east-1"



### C. Perform real-time inference

In [52]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://{file_name} --content-type $content_type --region $sagemaker_session.boto_region_name $output_file_name

{
    "ContentType": "text/html; charset=utf-8",
    "InvokedProductionVariant": "AllTraffic"
}


### D. Visualize output

In [53]:
output_file = "data/output/output.txt"  # Path to the output file

# Read the contents of the output file
with open(output_file, "r") as file:
    output_data = file.read()

# Print the output
print(output_data)


{"image": "..."}


### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
# upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

In [ ]:
# Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path

<font color='red'>For Seller to update: Add code that displays output generated by the batch transform job available in S3. 
This output must match the output available in data/output/batch folder.</font>

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.



## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/aws_marketplace|curating_aws_marketplace_listing_and_sample_notebook|ModelPackage|Sample_Notebook_Template|title_of_your_product-Model.ipynb)
